This code contains modules for processing raw behavioral videos to turn them into split view (bottom and side view) final videos for video scoring and pose estimation analysis (i.e. SLEAP). 
This script is written assuming the raw video files are in the same directory as the current script. Make a copy of this into your raw videos folder. 
*Requires ffmpeg 
*Please run these modules in order for the best results!

Activate conda environment with ffmpeg installed

In [ ]:
#conda activate simba


Note: you may need to restart the kernel to use updated packages.


Import relevant packages

In [1]:
import os
import subprocess

Convert avi to mp4 videos using GPU and output in current directory 

In [5]:
#define path to the directory containing .avi files
video_dir = r"Z:\20250711_FDMelittin_F_MA_KC\videos\raw\feeding" 
# Ensure the 'converted' directory exists or create one if it doesn't
output_dir = os.path.join(video_dir, 'converted')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Iterate over all .avi files in the specified directory
for file in os.listdir(video_dir):
    if file.endswith('.avi'):
        # Construct the file name for the converted video
        input_file = os.path.join(video_dir, file)
        output_file = os.path.join(output_dir, os.path.splitext(file)[0] + '.mp4')

        # Construct the ffmpeg command with NVENC
        convert = [
            'ffmpeg', '-i', input_file, '-c:v', 'h264_nvenc', '-pix_fmt', 'yuv420p',
            '-preset', 'medium', '-crf', '23', output_file
        ]

        # Execute the command
        subprocess.run(convert)

print("Conversion complete.")

Conversion complete.


Match video durations between bottom and side view videos if frame rate was optimized for exposure
*you will need to run this for each side view video

In [ ]:
# Step 1: Get the duration of the side view video
side_view_video = r"Y:\2025-02_FoodDep_Amanda\videos\raw\converted\20250130postCAP_front_rotated.mp4"
bottom_view_video = r"Y:\2025-02_FoodDep_Amanda\videos\raw\converted\20250130postCAP_left.mp4"
output_video = r"Y:\2025-02_FoodDep_Amanda\videos\raw\converted\20250130postCAP_front_rotated_adjusted.mp4"


side_view_duration_cmd = [
    "ffprobe",
    "-v", "error",
    "-show_entries", "format=duration",
    "-of", "default=noprint_wrappers=1:nokey=1",
    side_view_video
]
side_view_duration = float(subprocess.run(side_view_duration_cmd, stdout=subprocess.PIPE, universal_newlines=True).stdout.strip())

# Step 2: Get the duration of the bottom view video
bottom_view_duration_cmd = [
    "ffprobe",
    "-v", "error",
    "-show_entries", "format=duration",
    "-of", "default=noprint_wrappers=1:nokey=1",
    bottom_view_video
]
bottom_view_duration = float(subprocess.run(bottom_view_duration_cmd, stdout=subprocess.PIPE, universal_newlines=True).stdout.strip())

# Step 3: Calculate the duration ratio
duration_ratio = bottom_view_duration / side_view_duration

# Step 4: Adjust the side view video duration using ffmpeg
ffmpeg_cmd = [
    "ffmpeg",
    "-i", side_view_video,
    "-filter_complex", f"[0:v]setpts=PTS*{duration_ratio}[v]",
    "-map", "[v]",
    output_video
]

subprocess.run(ffmpeg_cmd)

Scale video
*use this if your bottom view and side view videos are not the same resolution; having the same resolution is required for concatenation and will minimize distortions when you create a split view of both videos

In [ ]:
file = r"C:\Users\LabAdmin\data_server\2025-01_FoodDepM_Amanda\videos\raw\20250109_220-2_FoodDepM_AN\converted\postCAP_right_adjusted.mp4"
output_file = r"C:\Users\LabAdmin\data_server\2025-01_FoodDepM_Amanda\videos\raw\20250109_220-2_FoodDepM_AN\converted\postCAP_right_adjusted_scaled.mp4"
scale_command= ['ffmpeg', '-i', file, '-s', '1920x1080', output_file]

# Execute the command
subprocess.run(scale_command)

Rotate video
*use this if the camera was tilted

In [ ]:
ffmpeg -hwaccel cuda -i "Y:\2025-02_FoodDep_Amanda\videos\raw\20250130postCAP_front.avi" -vf "rotate=-10*(PI/180)" -c:v h264_nvenc -preset p7 -qp 0 -c:a copy "Y:\2025-02_FoodDep_Amanda\videos\raw\20250130postCAP_front_rotated.avi"

Concatenate videos 
*run this if your experiment was recorded over two different video files

In [ ]:
def concatenate_videos(video_files):
    if not video_files:
        print("Error: No video files provided.")
        return
    
    # Extract directory and filename of the first input file
    first_file = video_files[0]
    first_name, ext = os.path.splitext(os.path.basename(first_file))
    output_file = os.path.join(os.path.dirname(first_file), f"{first_name}_concat{ext}")

    # Create a temporary file list
    file_list = os.path.join(os.path.dirname(first_file), "file_list.txt")

    with open(file_list, "w") as f:
        for video in video_files:
            f.write(f"file '{video}'\n")

    # Run FFmpeg command
    command = ["ffmpeg", "-f", "concat", "-safe", "0", "-i", file_list, "-c", "copy", output_file]

    try:
        subprocess.run(command, check=True)
        print(f"Concatenation successful! Output saved as {output_file}")
    except subprocess.CalledProcessError as e:
        print(f"Error during concatenation: {e}")
    
    # Clean up temporary file list
    os.remove(file_list)
video_files = [r"Z:\whiteplaceholder.mp4", r"Y:\2025-01_FoodDepM_Amanda\videos\raw\20250109_220-2_FoodDepM_AN\converted\temp\postCAP_left2_adjusted.mp4"]
concatenate_videos(video_files) #first raw string input is the first video, second is the second video i.e. the video to concatenate to the end of the first input


Your videos are now ready to be cropped and trimmed using simba
*create seperate folders to store your bottomView and sideView videos for the next step

Create split view video
*After cropping and trimming videos to the same length, you can now create a split view video with bottom view on the left and side view on the left
*This script pairs the videos based on the index of the videos in your side view and bottom view folders after sorting by alphabetical and numerical order

In [ ]:
# Define the paths to the folders
bottom_view_folder = r"Z:\20250711_FDMelittin_F_MA_KC\videos\final\postPain\bottomView\temp"
side_view_folder = r"Z:\20250711_FDMelittin_F_MA_KC\videos\final\postPain\sideView\temp"
output_folder = r"Z:\20250711_FDMelittin_F_MA_KC\videos\final\postPain"

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# List all files in the bottomView and sideView folders
bottom_videos = sorted([f for f in os.listdir(bottom_view_folder) if f.endswith('.mp4')])
side_videos = sorted([f for f in os.listdir(side_view_folder) if f.endswith('.mp4')])

# Pair the videos based on the filenames
for bottom_video, side_video in zip(bottom_videos, side_videos):
    bottom_video_path = os.path.join(bottom_view_folder, bottom_video)
    side_video_path = os.path.join(side_view_folder, side_video)
    output_video_name = bottom_video.replace('_bottom', '_combined')
    output_video_path = os.path.join(output_folder, output_video_name)
    
    subprocess.run([
        'ffmpeg', '-i', bottom_video_path, '-i', side_video_path,
        #'-t', '600',  # Trim to the first 10 minutes (600 seconds)
        '-filter_complex', '[1:v][0:v]scale2ref[1v][0v];[0v][1v]hstack', #compose bottom view (l) and side view (R) side by side, resize side view video to dimensions of bottom view
        '-c:v', 'h264_nvenc', output_video_path
    ])